In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/sample_submission.csv


In [30]:
test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
train = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
sample = test[:500]
train.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [32]:
new_sample = train[train['keyword'].notnull()]
new_sample.head()

,id,keyword,location,text
15,46,ablaze,London,Birmingham Wholesale Market is ablaze BBC News...
16,47,ablaze,Niall's place | SAF 12 SQUAD |,@sunkxssedharry will you wear shorts for race ...
17,51,ablaze,NIGERIA,#PreviouslyOnDoyinTv: Toke MakinwaÛªs marriag...
18,58,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http:/...
19,60,ablaze,"Los Angeles, Califnordia",PSA: IÛªm splitting my personalities.\n\n?? t...


**import libraries**
first things first import all the needed libraries here

In [6]:
import pandas as pd
import numpy as np
import seaborn as sn
from matplotlib import pyplot as plt
from string import punctuation
import re
import nltk

In [11]:
nltk.download('wordnet')

[nltk_data] Error loading wordnet: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


False

In [15]:
from nltk.stem import WordNetLemmatizer
lemm = WordNetLemmatizer()

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn
from sklearn.preprocessing import OneHotEncoder

stopwords = nltk.corpus.stopwords.words('english')


In [16]:
class TweetTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        new_X = X.copy()
        new_X['no_punct_tweet'] = new_X.apply(lambda x: "".join([word.lower() for word in x['text'] if word not in punctuation]), axis=1)
#         new_X['tokenize_tweet'] = new_X['no_punct_tweet'].apply(lambda x: re.split(r"\W+", x), axis = 1)
#         new_X['tokenize_tweet'] = new_X.apply(lambda x: re.split(r"\W+", x['no_punct_tweet']), axis = 1)
        new_X['tokenize_tweet'] = new_X.apply(lambda x: nltk.word_tokenize(x['no_punct_tweet']), axis=1)
        new_X['no_stopwords_tweet'] = new_X.apply(lambda x: [word for word in x['tokenize_tweet'] if word not in stopwords], axis=1)
        new_X['lemm'] = new_X.apply(lambda x: [lemm.lemmatize(word) for word in x['no_stopwords_tweet']], axis=1)
        return new_X

In [33]:
tt = TweetTransformer()
tt.fit(new_sample)
ttt = tt.transform(new_sample)

In [34]:
ttt

,id,keyword,location,text,no_punct_tweet,tokenize_tweet,no_stopwords_tweet,lemm
15,46,ablaze,London,Birmingham Wholesale Market is ablaze BBC News...,birmingham wholesale market is ablaze bbc news...,"[birmingham, wholesale, market, is, ablaze, bb...","[birmingham, wholesale, market, ablaze, bbc, n...","[birmingham, wholesale, market, ablaze, bbc, n..."
16,47,ablaze,Niall's place | SAF 12 SQUAD |,@sunkxssedharry will you wear shorts for race ...,sunkxssedharry will you wear shorts for race a...,"[sunkxssedharry, will, you, wear, shorts, for,...","[sunkxssedharry, wear, shorts, race, ablaze]","[sunkxssedharry, wear, short, race, ablaze]"
17,51,ablaze,NIGERIA,#PreviouslyOnDoyinTv: Toke MakinwaÛªs marriag...,previouslyondoyintv toke makinwaûªs marriage ...,"[previouslyondoyintv, toke, makinwaûªs, marri...","[previouslyondoyintv, toke, makinwaûªs, marri...","[previouslyondoyintv, toke, makinwaûªs, marri..."
18,58,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http:/...,check these out httptcoroi2nsmejj httptco3tj8z...,"[check, these, out, httptcoroi2nsmejj, httptco...","[check, httptcoroi2nsmejj, httptco3tj8zjin21, ...","[check, httptcoroi2nsmejj, httptco3tj8zjin21, ..."
19,60,ablaze,"Los Angeles, Califnordia",PSA: IÛªm splitting my personalities.\n\n?? t...,psa iûªm splitting my personalities\n\n techi...,"[psa, iûªm, splitting, my, personalities, tec...","[psa, iûªm, splitting, personalities, techies...","[psa, iûªm, splitting, personality, techie, f..."
...,...,...,...,...,...,...,...,...
3247,10806,wrecked,Seattle Washington,RT CNBC '3 words from Disney CEO Bob Iger wrec...,rt cnbc 3 words from disney ceo bob iger wreck...,"[rt, cnbc, 3, words, from, disney, ceo, bob, i...","[rt, cnbc, 3, words, disney, ceo, bob, iger, w...","[rt, cnbc, 3, word, disney, ceo, bob, iger, wr..."
3248,10807,wrecked,Acey mountain islanddåÇTorontoåÈ,Smackdown tyme this should put me in a good mo...,smackdown tyme this should put me in a good mo...,"[smackdown, tyme, this, should, put, me, in, a...","[smackdown, tyme, put, good, mood, since, got,...","[smackdown, tyme, put, good, mood, since, got,..."
3249,10816,wrecked,los angeles,@thrillhho jsyk I haven't stopped thinking abt...,thrillhho jsyk i havent stopped thinking abt r...,"[thrillhho, jsyk, i, havent, stopped, thinking...","[thrillhho, jsyk, havent, stopped, thinking, a...","[thrillhho, jsyk, havent, stopped, thinking, a..."
3250,10820,wrecked,"Brussels, Belgium",@stighefootball Begovic has been garbage. He g...,stighefootball begovic has been garbage he got...,"[stighefootball, begovic, has, been, garbage, ...","[stighefootball, begovic, garbage, got, wrecke...","[stighefootball, begovic, garbage, got, wrecke..."


In [36]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer(transformers=[('cat', OneHotEncoder(), [1])])
transformer.fit_transform(new_sample)
transformer

ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('cat',
                                 OneHotEncoder(categorical_features=None,
                                               categories=None, drop=None,
                                               dtype=<class 'numpy.float64'>,
                                               handle_unknown='error',
                                               n_values=None, sparse=True),
                                 [1])],
                  verbose=False)

In [ ]:
transformer.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()


In [ ]:
# no_punt = str.maketrans('', '', punctuation)

# def remove_punctuation(text):
#     no_punct_text = "".join([char for char in text if char not in punctuation])
#     return no_punct_text

# new = test
# new['text'] = new['text'].apply(lambda x: remove_punctuation(x.lower()))
# new.head()